In [177]:
import cv2 as cv 
import numpy as np
import os
import matplotlib.pyplot as plt
import copy

In [178]:
class Line:
    """
    Store a line based on the two points.
    """
    def __init__(self, point_1, point_2):
        self.point_1 = point_1
        self.point_2 = point_2
        

class Point:
    def __init__(self, x, y):
        self.x = np.int32(np.round(x))
        self.y = np.int32(np.round(y))
    
    def get_point_as_tuple(self):
        return (self.x, self.y)
    
        
class Patch:
    """
    Store information about each item (where the mark should be found) in the table. 
    """
    def __init__(self, image_patch, x_min, y_min, x_max, y_max, line_idx, column_idx):
        self.image_patch = image_patch
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max
        self.line_idx = line_idx
        self.column_idx = column_idx
        self.domino_value = -1 #no domino
    
    def set_domino_value(self, domino_value: int):
        assert any([domino_value == i-1 for i in range(8)])
        self.domino_value = domino_value

In [179]:
def show_image(image, window_name='image', timeout=0):
    """
    :param timeout. How many seconds to wait untill it close the window.
    """
    cv.imshow(window_name, cv.resize(image, None, fx=0.6, fy=0.6))
    cv.waitKey(timeout)
    cv.destroyAllWindows()

def draw_lines(image, lines: list[Line], timeout: int = 0, color: tuple = (0, 0, 255),
               return_drawing: bool = False, window_name: str = 'window'):
    """
    Plots the lines into the image.
    :param image.
    :param lines.
    :param timeout. How many seconds to wait untill it close the window.
    :param color. The color used to draw the lines
    :param return_drawing. Use it if you want the drawing to be return instead of displayed.
    :return None if return_drawing is False, otherwise returns the drawing.
    """
    drawing = image.copy()
    if drawing.ndim == 2:
        drawing = cv.cvtColor(drawing, cv.COLOR_GRAY2BGR)
    for line in lines: 
        cv.line(drawing, line.point_1.get_point_as_tuple(), line.point_2.get_point_as_tuple(), color, 2, cv.LINE_AA)
        
    if return_drawing:
        return drawing
    else:
        show_image(drawing, window_name=window_name, timeout=timeout)

In [180]:
def get_patches(lines: list[Line], columns: list[Line], image, show_patches: bool = False) -> list[Patch]:
    """
    It cuts out each box from the table defined by the lines and columns.
    :param lines. The lines that difine the table.
    :param columns. The columns that difine the table.
    :param image. The image containing the table.
    :param show_patches. Determine if the patches will be drawn on the image or not.
    :return : A list with all boxes in the table.
    """
    
    def crop_patch(image_, x_min, y_min, x_max, y_max):
        """
        Crops the bounding box represented by the coordinates.
        """
        return image_[y_min: y_max, x_min: x_max].copy()
    
    def draw_patch(image_, patch: Patch, color: tuple = (255, 0, 255)):
        """
        Draw the bounding box corresponding to the patch on the image.
        """
        cv.rectangle(image_, (patch.x_min, patch.y_min), (patch.x_max, patch.y_max), color=color, thickness=5)
    
    assert image.ndim == 2
    if show_patches: 
        image_color = np.dstack((image, image, image))
  
    lines.sort(key=lambda line: line.point_1.y)
    columns.sort(key=lambda column: column.point_1.x)
    patches = []
    step = 0
    for line_idx in range(len(lines) - 1):
        for col_idx in range(len(columns) - 1):
            current_line = lines[line_idx]
            next_line = lines[line_idx + 1] 
            
            y_min = current_line.point_1.y + step
            y_max = next_line.point_1.y - step
            
            current_col = columns[col_idx]
            next_col = columns[col_idx + 1]
            x_min = current_col.point_1.x + step 
            x_max = next_col.point_1.x - step
            
            patch = Patch(image_patch=crop_patch(image,  x_min, y_min, x_max, y_max),
                          x_min=x_min, y_min=y_min, x_max=x_max, y_max=y_max,
                          line_idx=line_idx, column_idx=col_idx)
            
            if show_patches:
                draw_patch(image_color, patch)
            
            patches.append(patch)
            
    if show_patches:
        show_image(image_color, window_name='patches', timeout=0)
    return patches

In [181]:
def find_patches_template(image, template, show_intermediate_results: bool = False):
    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    template_gray = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
    query_image_edges = cv.Canny(image_gray, 200, 250) #the params were found by trial and error
    template_image_edges = cv.Canny(template_gray, 200, 250)
    
    if show_intermediate_results: 
        show_image(query_image_edges, window_name='query_image_edges', timeout=0)
        show_image(template_image_edges, window_name='template_image_edges', timeout=0)
        
    alligned_image = image
    
    left_corner = Point(x=0, y=0)
    right_corner = Point(x=len(template), y=len(template))

    step_x = (right_corner.x - left_corner.x) / 15
    step_y = (right_corner.y - left_corner.y) / 15
    
    def get_vertical_and_horizontal_lines(left_corner: Point, step_x_, step_y_): 
        vertical_lines = []
        horizontal_lines = []
        
        for i in range(16):
            xmin = np.uint64(left_corner.x + i * step_x_)
            line = Line(Point(x = xmin, y = 0), Point(x = xmin, y = 5000)) 
            vertical_lines.append(line)
        for i in range(16):
            ymin = np.uint64(left_corner.y + i * step_y_)
            line = Line(Point(x = 0, y = ymin), Point(x = 5000, y = ymin))
            horizontal_lines.append(line)

        return vertical_lines, horizontal_lines
     
    vertical_lines, horizontal_lines = get_vertical_and_horizontal_lines(left_corner, step_x, step_y) 
    
    alligned_image_gray = cv.cvtColor(alligned_image, cv.COLOR_BGR2GRAY)
    patches = get_patches(horizontal_lines, vertical_lines, alligned_image_gray, 
                          show_patches=show_intermediate_results)
    
    return patches

In [182]:
class DominoClassifier:
    """
    Classifier that determines the value of the domino
    """
    def __init__(self):
        self.whites = [0 for _ in range(6)]
        self.blacks = [0 for _ in range(6)]
    
    def classify(self, patch: Patch) -> int:
        """
        Uses Hough circles to determine the domino value
        """
        def is_domino(patch):
            """
            Receive a (greyscale) Patch and return True if it is classified as a domino and False otherwise
            """
            _, thresh = cv.threshold(patch, 185, 255, cv.THRESH_BINARY) #thresh value found through trial and error

            sum_whites = np.sum(thresh == 255)
            sum_blacks = np.sum(thresh == 0)

            if sum_whites >= min(self.whites) and sum_whites <= max(self.whites) and sum_blacks >= min(self.blacks) and sum_blacks <= max(self.blacks):
                return True
            
            return False

        if not is_domino(patch):
            return -1

        circles = cv.HoughCircles(patch, cv.HOUGH_GRADIENT, 1, patch.shape[0] / 8, param1=100, param2=30, minRadius=5, maxRadius=15)
        if circles is not None:
            circles = np.uint16(np.around(circles))
            # cv.imshow('patch', patch)
            # cv.waitKey(0)
            # cv.destroyAllWindows()
            # for i in circles[0,:]:
            #     # draw the outer circle
            #     cv.circle(patch_copy,(i[0],i[1]),i[2],(0,255,0),2)
            #     # draw the center of the circle
            #     cv.circle(patch_copy,(i[0],i[1]),2,(0,0,255),3)

            # cv.imshow('found circles', patch_copy)
            # cv.waitKey(0)
            # cv.destroyAllWindows()
            return len(circles[0,:]) # number of circles
        
        return -1 # for false positives
        
    def get_domino_whites_and_blacks(self, dominoes_path='dominoes/'):   
        domino_filenames = [dominoes_path + f for f in os.listdir(dominoes_path) if f.endswith('.jpg')]
        whites = []
        blacks = []

        for domino_filename in domino_filenames:
            domino = cv.imread(domino_filename)
            domino = cv.cvtColor(domino, cv.COLOR_BGR2GRAY)

            _, thresh = cv.threshold(domino, 185, 255, cv.THRESH_BINARY)

            whites.append(np.sum(thresh == 255)) 
            blacks.append(np.sum(thresh == 0))

        self.whites = whites
        self.blacks = blacks
        return whites, blacks

def classify_patches_with_domino_classifier(patches: list[Patch], dominoes_path: str = 'dominoes/') -> None:
    """
    Receive the patches and classify if the patch contains an 'X' or not.
    :param patches.
    :return None
    """
    domino_classifier = DominoClassifier()
    domino_classifier.get_domino_whites_and_blacks(dominoes_path)
    for line in patches:
        for column in line:
            column.set_domino_value(domino_classifier.classify(column.image_patch))

In [183]:
score_to_domino_dict = {
    0: 1,
    1: 2,
    2: 3,
    3: 4,
    4: 5,
    5: 6,
    6: 0,
    7: 2,
    8: 5,
    9: 3,
    10: 4,
    11: 6,
    12: 2,
    13: 2,
    14: 0,
    15: 3,
    16: 5,
    17: 4,
    18: 1,
    19: 6,
    20: 2,
    21: 4,
    22: 5,
    23: 5,
    24: 0,
    25: 6,
    26: 3,
    27: 4,
    28: 2,
    29: 0,
    30: 1,
    31: 5,
    32: 1,
    33: 3,
    34: 4,
    35: 4,
    36: 4,
    37: 5,
    38: 0,
    39: 6,
    40: 3,
    41: 5,
    42: 4,
    43: 1,
    44: 3,
    45: 2,
    46: 0,
    47: 0,
    48: 1,
    49: 1,
    50: 2,
    51: 3,
    52: 6,
    53: 3,
    54: 5,
    55: 2,
    56: 1,
    57: 0,
    58: 6,
    59: 6,
    60: 5,
    61: 2,
    62: 1,
    63: 2,
    64: 5,
    65: 0,
    66: 3,
    67: 3,
    68: 5,
    69: 0,
    70: 6,
    71: 1,
    72: 4,
    73: 0,
    74: 6,
    75: 3,
    76: 5,
    77: 1,
    78: 4,
    79: 2,
    80: 6,
    81: 2,
    82: 3,
    83: 1,
    84: 6,
    85: 5,
    86: 6,
    87: 2,
    88: 0,
    89: 4,
    90: 0,
    91: 1,
    92: 6,
    93: 4,
    94: 4,
    95: 1,
    96: 6,
    97: 6,
    98: 3,
    99: 0
}

In [184]:
def get_game_move_to_patches(images_path = 'aligned_train/'):
    template = cv.imread(images_path + 'aligned_1_01.jpg')
    game_to_patches = {}

    img_filenames = [images_path + f for f in os.listdir(images_path) if f.endswith('.jpg') and f.startswith('aligned')]
    for img_filename in img_filenames:
        original_image = cv.imread(img_filename) 
        patches = find_patches_template(original_image, template)
        patches = np.reshape(patches, (15,15))
        game, move = img_filename.split('/')[-1].split('.')[0].split('_')[1:3]
        game_to_patches[(game, move)] = patches
    return game_to_patches

In [198]:
def whose_move(game, moves_path = 'train/regular_tasks/'):
    player_to_moves_filename = moves_path + game + '_moves.txt' 

    with open(player_to_moves_filename, 'r') as f:
        lines = f.readlines()

    move_to_player_dict = {}
    for line in lines:
        line_split = line.split(' ')
        move = line_split[0].split('.')[0].split('_')[-1]
        if line_split[-1] == 'player1':
            move_to_player_dict[move] = 'player1'
        elif line_split[-1] == 'player2':
            move_to_player_dict[move] = 'player2'
    
    return move_to_player_dict


def get_scores(images_path = 'aligned_train/', out_path = 'scores/', moves_path = 'train/regular_tasks/'):
    if not os.path.exists(out_path):
        os.mkdir(out_path)

    game_to_patches = get_game_move_to_patches(images_path)

    games_to_moves = {} # better data structure to store data
    for item in game_to_patches:
        if item[0] not in games_to_moves:
            games_to_moves[item[0]] = []
        games_to_moves[item[0]].append({item[1]: game_to_patches[(item[0], item[1])]})

    for game in games_to_moves.keys():
        game_to_player_moves = whose_move(game, moves_path)
        print(game_to_player_moves)
        player1_position = 0
        player2_position = 0
        
        player1_score = 0
        player2_score = 0

        domino_list = []
        for move_to_patches in games_to_moves[game]:
            individual_move = [item for item in move_to_patches.items()][0]

            move = individual_move[0]

            classify_patches_with_domino_classifier(game_to_patches[(game, move)])
            with open(out_path + game + '_' + move + '.txt', 'w') as f:
                for i, line in enumerate(game_to_patches[(game, move)]):
                    for j, column in enumerate(line):
                        if column.domino_value != -1:
                            position = str(i + 1) + str(chr(ord('@') + j + 1))
                            value = column.domino_value

                            if position not in domino_list:
                                output = position + ' ' + str(value) 
                                domino_list.append(position)
                                f.write(output + '\n')

images_path = 'aligned_train/'
out_path = 'scores/'
get_scores(images_path, out_path)

{}
{}
{}
{}
{}
